In [ ]:
import pyawr.mwoffice as mwo
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation 

'***PICK ONE***'
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
%matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
# %matplotlib tk 

In [ ]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #Invoke circuit simulator analysis

In [ ]:
graph_name = awrde.Project.Graphs.Item(2).Name # graphs are named in MWO. This is how you get the name.
graph = awrde.Project.Graphs(graph_name) # get the graph object by name
meas = graph.Measurements[0] # get the measurement object (e.g the plot of S11) from the graph Graph(graph_name)

In [ ]:
awrde.Project.Frequencies.Clear() # clear the frequencies specified for the project
awrde.Project.Frequencies.AddMultiple(np.linspace(4e9, 8e9, 10000)) # add the frequencies in the passed prequency array

In [ ]:
'''This entire block is nonsense'''

sample_circ_name = 'Inductor_Subcircuit'
sample_circ_param_vals = {
    'CAP': awrde.Project.Schematics(sample_circ_name).Elements(1).Parameters(2).ValueAsDouble,
    'IND': awrde.Project.Schematics(sample_circ_name).Elements(2).Parameters(2).ValueAsDouble,
    'RES': awrde.Project.Schematics(sample_circ_name).Elements(3).Parameters(2).ValueAsDouble
}

print(sample_circ_param_vals)

def bad_set_circ_params(**kwargs): # dictionary keys should match keys in sample_circ_params
    print("About to set:" + kwargs.keys().__str__())

    sample_circ_params = {    
        'CAP': awrde.Project.Schematics(sample_circ_name).Elements(1).Parameters(2),
        'IND': awrde.Project.Schematics(sample_circ_name).Elements(2).Parameters(2),
        'RES': awrde.Project.Schematics(sample_circ_name).Elements(3).Parameters(2)
    }

    new_circ_param_vals = {**kwargs, **sample_circ_param_vals}
    print(new_circ_param_vals)

    for param in sample_circ_params.items():
        awrde.Project.Schematics(sample_circ_name).Elements(1).Parameters(2).ValueAsDouble = new_circ_param_vals.get(param[0])
        print(new_circ_param_vals.get(param[0]))

# set_circ_params(CAP=0)
    
    # awrde.Project.Schematics(schem_name).Elements(1).Parameters(2).ValueAsDouble(new_circ_params.values[0])
    # awrde.Project.Schematics(schem_name).Elements(2).Parameters(2).ValueAsDouble(new_circ_params.values[1])
    # awrde.Project.Schematics(schem_name).Elements(3).Parameters(2).ValueAsDouble(new_circ_params.values[2])

In [ ]:
'''Set subcircuit paramter values'''

sample_circ_name = 'Inductor_Subcircuit'
sample_circ_param_vals = { # dictionary of subcircuit element parameter values (e.g. the value of the capacitor (element's) capacitance (parameter.))
    'CAP': awrde.Project.Schematics(sample_circ_name).Elements(1).Parameters(2).ValueAsDouble,
    'IND': awrde.Project.Schematics(sample_circ_name).Elements(2).Parameters(2).ValueAsDouble,
    'RES': awrde.Project.Schematics(sample_circ_name).Elements(3).Parameters(2).ValueAsDouble
}
sample_circ_params = { # dictionary of subcircuit component 
    'CAP': awrde.Project.Schematics(sample_circ_name).Elements(1).Parameters(2),
    'IND': awrde.Project.Schematics(sample_circ_name).Elements(2).Parameters(2),
    'RES': awrde.Project.Schematics(sample_circ_name).Elements(3).Parameters(2)
}

def set_circ_params(**kwargs): # dictionary keys should match keys in sample_circ_params. If you attempt to pass a parameter that doesn't exit, MWO will throw an error.
    print("About to set:" + kwargs.__str__())

    new_circ_param_vals = {**sample_circ_param_vals, **kwargs} # in the case of duplicate keys, only the later key-value pair is preserved
    print("DEBUG: new parameters:" + new_circ_param_vals.__str__())

    for i, value in enumerate(new_circ_param_vals.values()):
        list(sample_circ_params.values())[i].ValueAsDouble = value # in python3, dict.keys(), .values(), and .items() return dynamically changing view objects, but not the objects themselves. Hence, list().

        # print("DEBUG: " + list(sample_circ_param_vals)[i] + " set to " + str(value))


In [ ]:
'''Specifiying the sweep of a parameter'''

# -----USER-SETTABLE:-----
sample_circ_param_bounds = {
    # If a paramter is missing it will not be reset. 
    #'COMPONENT' : (lower_bound, upper_bound)    
    'CAP': (0,1),
    # 'IND': (0,1),
    # 'RES': (1000000000,1000000000)
}
resolution = 10 # sets the number of parameter values to simulate between the bounds (i.e. the resolution of the simulation.)
#-------------------------

sample_circ_param_arrays = dict.fromkeys(sample_circ_param_bounds)

keys = sample_circ_param_arrays.keys()
bnds = sample_circ_param_bounds.values()
arrs = list(np.linspace(*bnds_tuple, resolution) for bnds_tuple in bnds)

sample_circ_param_arrays.update(zip(keys, arrs))

print("DEBUG: " + sample_circ_param_arrays.__str__())
 

In [ ]:
'''Running the sumulation with set parameters'''

# FIXME: This loop actually changes all the elements simultaneously. Add some logic to do it iteratively...
for ind in range(resolution):
    indiv_vals = list(arr[ind] for arr in arrs)
    ele_val_dict = dict(zip(keys, indiv_vals))
    # FIXME: This is probably not great, first creating a dictionary then unpacking it immediately...
    set_circ_params(**ele_val_dict)

    print('DEBUG: ' + ele_val_dict.__str__())
    